In [1]:
#1 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [2]:
#2 定义超参数
BATCH_SIZE = 16 #每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")# 是否用GPU训练
EPOCHS = 10 #训练数据集的轮数  比如一轮10万张，10轮60万，不断训练这个模型


In [3]:
#3 构建pipline，对图像做处理
pipline = transforms.Compose([
    transforms.ToTensor(),#将图片转化成Tensor
    transforms.Normalize((0.1307,),(0.3081,))#正则化：降低模型的复杂度
])

In [4]:
#4下载，加载数据
from torch.utils.data import DataLoader

#下载数据集
train_set = datasets.MNIST("data", train=True, download=True,transform=pipline) # 把数据集放在根目录
test_set = datasets.MNIST("data", train=False, download=True,transform=pipline) 

#加载数据
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)

test_loader= DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [5]:
# #插入代码显示MNIST中的图片,看一看而已
# with open("D:\\专门用于科研的文件夹\\轻松学pytorch MNIST手写体识别\\data\\MNIST\\raw\\train-images-idx3-ubyte","rb") as f:
#     file= f.read()

In [6]:
# image1=[int(str(item).encode('ascii'),16) for item in file[16: 16+784]]
# print(image1)


In [7]:
#5 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)#1：灰度的通道， 10：输出通道， 5：kernel
        self.conv2 = nn.Conv2d(10,20,3)#10：输入通道 ， 20： 输出通道 3：kernel
        self.fc1 = nn.Linear(20*10*10,500)#20*10*10:输入通道， 500：输出通道
        self.fc2 = nn.Linear(500,10)#500: 输入通道 10： 输出通道
    
    def forward(self,x):
        input_size=x.size(0)#batch_size x 1 x 28 x 28 
        x=self.conv1(x) #输入：batch_size*1*28*28  输出：batch*10*24*24(28-5+1=24)
        x=F.relu(x)#保持shape不变 输出：batch*10*24*24
        x=F.max_pool2d(x,2,2)#输入：batch*10*24*24 输出：batch*10*12*12
        x=self.conv2(x)#输入：batch*10*12*12 输出：batch*20*10*10（12-3+1=10）
        x=F.relu(x)
        
        x=x.view(input_size,-1) #拉平， -1 ：自动计算维度，20*10*10=2000
        
        x= self.fc1(x) #输入：batch*2000 输出：batch*500
        x=F.relu(x) #保持shape不变
        x= self.fc2(x) #输入：batch*500 输出： batch*10
        output = F.log_softmax(x,dim=1)#计算分类后每个数字的概率值
        return output
        

In [8]:
#6 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())


In [9]:
#7 定义训练方法
def train_model(model,device,train_loader,optimizer,epoch):
    #模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #把数据部署到device上去
        data,target = data.to(device),target.to(device)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output,target)
        #找到概率值最大的下标
        pred= output.max(1,keepdim=True) #pred=output.argmax(dim=1)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 ==0:
            print("Train Epoch:{}\t Loss:{:.6f}".format(epoch,loss.item()))
        
        

In [10]:
#8 定义测试方法
def test_model(model,device,test_loader):
    #模型验证
    model.eval()
    #正确率
    correct =0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad():#不会计算梯度， 也不会反向传播
        for data,target in test_loader:
            #部署到device上
            data,target=data.to(device),target.to(device)
            #测试数据
            output=model(data)
            #计算测试损失
            test_loss +=F.cross_entropy(output,target).item()
            #找到概率值最大的下标
            pred=output.max(1,keepdim=True)[1] #值，索引
            #累计正确率
            correct +=pred.eq(target.view_as(pred)).sum().item()
        test_loss /=len(test_loader.dataset)
        print("test --Averager loss :{:.4f},Accuracy:{:.3f}\n".format(test_loss,100.0*correct/len(test_loader.dataset)))

In [11]:
#9 调用 方法7和8
for epoch in range(1,EPOCHS +1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.298765
Train Epoch:1	 Loss:0.096877
test --Averager loss :0.0031,Accuracy:98.450

Train Epoch:2	 Loss:0.001326
Train Epoch:2	 Loss:0.016128
test --Averager loss :0.0026,Accuracy:98.770

Train Epoch:3	 Loss:0.001285
Train Epoch:3	 Loss:0.001398
test --Averager loss :0.0028,Accuracy:98.670

Train Epoch:4	 Loss:0.007158


KeyboardInterrupt: 